In [23]:
# Import Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, classification_report, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import warnings
import shap
import numpy as np
from art.attacks.poisoning import PoisoningAttackBackdoor, FeatureCollisionAttack
from art.estimators.classification import SklearnClassifier
from art.utils import to_categorical

sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

In [24]:
# Load the preprocessed data
data = pd.read_csv('Preprocessed_Data.csv')

# Rename columns to remove special characters
data.rename(columns={
    'Air temperature [K]': 'Air_temperature_K',
    'Process temperature [K]': 'Process_temperature_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
}, inplace=True)

# Define features and target
X = data[['Type','Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min']]
y = data['Machine failure']

In [25]:
# Initialize stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Apply SMOTE to oversample the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [26]:
# Define models with the best parameters
models = {
    'Random Forest': RandomForestClassifier(
        max_depth=20, 
        min_samples_split=2, 
        n_estimators=350, 
        random_state=42,
    ),
    'XGBoost': XGBClassifier(
        learning_rate=0.2, 
        max_depth=7, 
        n_estimators=350, 
        subsample=0.9, 
        random_state=42,
    ), 'Neural Network': MLPClassifier(
        activation='tanh', 
        hidden_layer_sizes=(50, 50), 
        solver='adam',
        max_iter=400,
        random_state=42
    )
}

# Label Flip Attack:

In [27]:
# Define the poisoning function for binary classification (flipping 0 to 1 or 1 to 0)
def label_flip_poisoning(X_train, y_train, poison_percentage, target_class):
    # Find the indices where y_train indicates the target class (failure or no failure)
    class_indices = np.where(y_train == target_class)[0]
    
    # Detaermine the number of labels to flip based on the poison percentage
    num_to_flip = int(poison_percentage * len(class_indices))
    
    # Randomly select indices to flip
    flip_indices = np.random.choice(class_indices, size=num_to_flip, replace=False)
    
    # Create poisoned labels by copying y_train
    y_train_poisoned = y_train.copy()
    
    # Flip the selected failure/no failure labels (if 1 -> 0, if 0 -> 1)
    y_train_poisoned[flip_indices] = 1 - target_class
    
    return X_train, y_train_poisoned

In [28]:
def identify_suspicious_labels(X_train, y_train, threshold=0.7, n_neighbors=3):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)

    # Ensure X_train is an array for KNN compatibility
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.values

    y_pred_knn = knn.predict(X_train)
    mismatches = (y_train != y_pred_knn)
    suspicious_idx = np.where(mismatches)[0]
    return suspicious_idx

In [29]:
# Define label correction
def correct_labels(X_train, y_train, suspicious_idx):
    corrected_labels = y_train.copy()
    corrected_labels[suspicious_idx] = 1 - y_train[suspicious_idx]  # Flip labels
    return corrected_labels

# Define label deletion
def delete_labels(X_train, y_train, suspicious_idx):
    X_train_cleaned = np.delete(X_train, suspicious_idx, axis=0)
    y_train_cleaned = np.delete(y_train, suspicious_idx, axis=0)
    return X_train_cleaned, y_train_cleaned

In [30]:
# Define the poisoning percentages to test
poison_percentages = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
target_classes = [0, 1]  # 0 for no failure, 1 for failure

# Initialize dictionaries to hold DataFrames for each metric, model, target class, and intervention type
accuracy_results = {'Model': [], 'Target': [], 'Intervention': []}
precision_results = {'Model': [], 'Target': [], 'Intervention': []}
recall_results = {'Model': [], 'Target': [], 'Intervention': []}
f1_results = {'Model': [], 'Target': [], 'Intervention': []}
fdr_results = {'Model': [], 'Target': [], 'Intervention': []}  # For Failure Detection Rate

# Add columns for each poisoning percentage in the results dictionaries
for poison_percentage in poison_percentages:
    accuracy_results[f'{int(poison_percentage * 100)}%'] = []
    precision_results[f'{int(poison_percentage * 100)}%'] = []
    recall_results[f'{int(poison_percentage * 100)}%'] = []
    f1_results[f'{int(poison_percentage * 100)}%'] = []
    fdr_results[f'{int(poison_percentage * 100)}%'] = []  # FDR results

# Add a column to store the mean of each metric across the poisoning percentages
accuracy_results['Mean'] = []
precision_results['Mean'] = []
recall_results['Mean'] = []
f1_results['Mean'] = []
fdr_results['Mean'] = []  # FDR Mean

In [31]:
# Define the interventions
interventions = ['No Intervention', 'Correction', 'Deletion']

# Loop through each model
for name, model in models.items():
    print(f'Model: {name}')
    
    # Loop through each target class (0 and 1)
    for target_class in target_classes:
        print(f'Target Class: {target_class}')
        
        # Loop through each intervention type (No Intervention, Correction, Deletion)
        for intervention in interventions:
            print(f'Intervention: {intervention}')
            
            # Store the model name, target class, and intervention type in the results dictionaries
            accuracy_results['Model'].append(name)
            precision_results['Model'].append(name)
            recall_results['Model'].append(name)
            f1_results['Model'].append(name)
            fdr_results['Model'].append(name)

            accuracy_results['Target'].append(target_class)
            precision_results['Target'].append(target_class)
            recall_results['Target'].append(target_class)
            f1_results['Target'].append(target_class)
            fdr_results['Target'].append(target_class)

            accuracy_results['Intervention'].append(intervention)
            precision_results['Intervention'].append(intervention)
            recall_results['Intervention'].append(intervention)
            f1_results['Intervention'].append(intervention)
            fdr_results['Intervention'].append(intervention)
            
            # Store a list to calculate the mean values later
            acc_values = []
            prec_values = []
            rec_values = []
            f1_values = []
            fdr_values = []
            
            # Loop through each poisoning percentage (including 0% for clean data)
            for poison_percentage in poison_percentages:
                print(f'Poison percentage: {poison_percentage}')

                # Poison the training data
                X_train_poisoned, y_train_poisoned = label_flip_poisoning(X_train_res, y_train_res, poison_percentage, target_class)
                
                # Apply label correction or deletion if needed
                if intervention == 'Correction':
                    suspicious_indices = identify_suspicious_labels(X_train_poisoned, y_train_poisoned)
                    y_train_poisoned[suspicious_indices] = y_train_res[suspicious_indices]  # Correct labels
                elif intervention == 'Deletion':
                    suspicious_indices = identify_suspicious_labels(X_train_poisoned, y_train_poisoned)
                    X_train_poisoned, y_train_poisoned = delete_labels(X_train_poisoned, y_train_poisoned, suspicious_indices)

                # Train the model on the (possibly corrected or cleaned) poisoned data
                model.fit(X_train_poisoned, y_train_poisoned)
                
                # Make predictions on the clean test set
                y_pred = model.predict(X_test)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                f1 = f1_score(y_test, y_pred, average='weighted')
                
                # Calculate Failure Detection Rate (FDR) for class 1 (failure)
                if target_class == 1:
                    true_failure = (y_test == 1)
                    true_positive_failures = ((y_pred == 1) & (y_test == 1)).sum()
                    fdr = true_positive_failures / true_failure.sum() if true_failure.sum() > 0 else 0
                else:
                    fdr = None  # FDR is only relevant for the failure class
                
                # Store the metrics based on the poison percentage for the current intervention and target class
                accuracy_results[f'{int(poison_percentage * 100)}%'].append(accuracy)
                precision_results[f'{int(poison_percentage * 100)}%'].append(precision)
                recall_results[f'{int(poison_percentage * 100)}%'].append(recall)
                f1_results[f'{int(poison_percentage * 100)}%'].append(f1)
                fdr_results[f'{int(poison_percentage * 100)}%'].append(fdr)

                # Store the metrics for mean calculation
                acc_values.append(accuracy)
                prec_values.append(precision)
                rec_values.append(recall)
                f1_values.append(f1)
                if fdr is not None:
                    fdr_values.append(fdr)

            # Calculate the mean of each metric across all poisoning percentages
            accuracy_results['Mean'].append(sum(acc_values) / len(acc_values))
            precision_results['Mean'].append(sum(prec_values) / len(prec_values))
            recall_results['Mean'].append(sum(rec_values) / len(rec_values))
            f1_results['Mean'].append(sum(f1_values) / len(f1_values))
            if fdr_values:
                fdr_results['Mean'].append(sum(fdr_values) / len(fdr_values))
            else:
                fdr_results['Mean'].append(None)

# Convert the results dictionaries to DataFrames
accuracy_df = pd.DataFrame(accuracy_results)
precision_df = pd.DataFrame(precision_results)
recall_df = pd.DataFrame(recall_results)
f1_df = pd.DataFrame(f1_results)
fdr_df = pd.DataFrame(fdr_results)  # FDR DataFrame

# Display the results
print("Accuracy Results:")
display(accuracy_df)

print("Precision Results:")
display(precision_df)

print("Recall Results:")
display(recall_df)

print("F1 Score Results:")
display(f1_df)

print("Failure Detection Rate (FDR) Results:")
display(fdr_df)

Model: Random Forest
Target Class: 0
Intervention: No Intervention
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Intervention: Correction
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Intervention: Deletion
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Target Class: 1
Intervention: No Intervention
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Intervention: Correction
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Intervention: Deletion
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.

,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,0,No Intervention,0.964000,0.955000,0.941000,0.908000,0.798667,0.459333,0.837667
1,Random Forest,0,Correction,0.964000,0.962000,0.957333,0.946667,0.901667,0.772667,0.917389
2,Random Forest,0,Deletion,0.956667,0.949667,0.937667,0.920667,0.783667,0.461667,0.835000
3,Random Forest,1,No Intervention,0.964000,0.969333,0.969000,0.974000,0.978000,0.969667,0.970667
4,Random Forest,1,Correction,0.964000,0.968000,0.969333,0.970667,0.974667,0.976333,0.970500
5,Random Forest,1,Deletion,0.956667,0.961000,0.965667,0.974667,0.973000,0.968000,0.966500
6,XGBoost,0,No Intervention,0.976667,0.957000,0.905667,0.807667,0.654000,0.477000,0.796333
7,XGBoost,0,Correction,0.976667,0.968000,0.953333,0.910667,0.829333,0.674667,0.885444
8,XGBoost,0,Deletion,0.960000,0.954000,0.930333,0.861333,0.683333,0.469667,0.809778
9,XGBoost,1,No Intervention,0.976667,0.973000,0.975000,0.972667,0.968333,0.973333,0.973167


Precision Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,0,No Intervention,0.972834,0.972153,0.971446,0.969036,0.966836,0.964168,0.969412
1,Random Forest,0,Correction,0.972834,0.972550,0.973798,0.971221,0.968968,0.966699,0.971012
2,Random Forest,0,Deletion,0.970718,0.969883,0.970174,0.970155,0.967853,0.964197,0.968830
3,Random Forest,1,No Intervention,0.972834,0.973846,0.971544,0.970182,0.976997,0.965503,0.971818
4,Random Forest,1,Correction,0.972834,0.974576,0.973229,0.973244,0.974195,0.973229,0.973551
5,Random Forest,1,Deletion,0.970718,0.972255,0.971735,0.974195,0.969559,0.959010,0.969579
6,XGBoost,0,No Intervention,0.979587,0.975175,0.968854,0.967514,0.964724,0.962185,0.969673
7,XGBoost,0,Correction,0.979587,0.977831,0.974012,0.970884,0.967689,0.967112,0.972853
8,XGBoost,0,Deletion,0.973750,0.974892,0.971992,0.966293,0.965150,0.962805,0.969147
9,XGBoost,1,No Intervention,0.979587,0.973387,0.973765,0.968726,0.961205,0.968867,0.970923


Recall Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,0,No Intervention,0.964000,0.955000,0.941000,0.908000,0.798667,0.459333,0.837667
1,Random Forest,0,Correction,0.964000,0.962000,0.957333,0.946667,0.901667,0.772667,0.917389
2,Random Forest,0,Deletion,0.956667,0.949667,0.937667,0.920667,0.783667,0.461667,0.835000
3,Random Forest,1,No Intervention,0.964000,0.969333,0.969000,0.974000,0.978000,0.969667,0.970667
4,Random Forest,1,Correction,0.964000,0.968000,0.969333,0.970667,0.974667,0.976333,0.970500
5,Random Forest,1,Deletion,0.956667,0.961000,0.965667,0.974667,0.973000,0.968000,0.966500
6,XGBoost,0,No Intervention,0.976667,0.957000,0.905667,0.807667,0.654000,0.477000,0.796333
7,XGBoost,0,Correction,0.976667,0.968000,0.953333,0.910667,0.829333,0.674667,0.885444
8,XGBoost,0,Deletion,0.960000,0.954000,0.930333,0.861333,0.683333,0.469667,0.809778
9,XGBoost,1,No Intervention,0.976667,0.973000,0.975000,0.972667,0.968333,0.973333,0.973167


F1 Score Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,0,No Intervention,0.967493,0.961484,0.952321,0.931108,0.862016,0.595146,0.878261
1,Random Forest,0,Correction,0.967493,0.966119,0.963434,0.955885,0.927164,0.844924,0.937503
2,Random Forest,0,Deletion,0.962154,0.957452,0.949908,0.939216,0.852240,0.597399,0.876395
3,Random Forest,1,No Intervention,0.967493,0.971219,0.970147,0.970918,0.973529,0.959053,0.968727
4,Random Forest,1,Correction,0.967493,0.970615,0.970998,0.971810,0.974422,0.972886,0.971371
5,Random Forest,1,Deletion,0.962154,0.965387,0.968192,0.974422,0.970653,0.958308,0.966519
6,XGBoost,0,No Intervention,0.977846,0.963563,0.929626,0.867883,0.761179,0.612425,0.852087
7,XGBoost,0,Correction,0.977846,0.971556,0.960877,0.933105,0.881776,0.776405,0.916928
8,XGBoost,0,Deletion,0.965163,0.961525,0.945669,0.901747,0.782867,0.605345,0.860386
9,XGBoost,1,No Intervention,0.977846,0.973189,0.974311,0.969822,0.962919,0.968656,0.971124


Failure Detection Rate (FDR) Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,0,No Intervention,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Random Forest,0,Correction,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Random Forest,0,Deletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Random Forest,1,No Intervention,0.725490,0.696078,0.627451,0.421569,0.382353,0.137255,0.498366
4,Random Forest,1,Correction,0.725490,0.735294,0.676471,0.656863,0.607843,0.421569,0.637255
5,Random Forest,1,Deletion,0.725490,0.735294,0.676471,0.607843,0.450980,0.147059,0.557190
6,XGBoost,0,No Intervention,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,XGBoost,0,Correction,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,XGBoost,0,Deletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,XGBoost,1,No Intervention,0.774510,0.617647,0.578431,0.421569,0.274510,0.343137,0.501634


In [32]:
# Function to plot a metric for a model across poisoning percentages, grouped by intervention type
def plot_metric_separate(metric_df, metric_name, model_name, target_class):
    plt.figure(figsize=(10, 6))

    # Filter the DataFrame for the selected target class and model
    filtered_df = metric_df[(metric_df['Target'] == target_class) & (metric_df['Model'] == model_name)]

    # Iterate through each intervention and plot its metric across poisoning percentages (excluding the mean column)
    for intervention in filtered_df['Intervention'].unique():
        # Filter by intervention type
        intervention_df = filtered_df[filtered_df['Intervention'] == intervention]

        # Get the values as numpy array, stripping the '%' symbol and converting to integers
        # Exclude the 'Mean' column from the x_values
        x_values = [int(col.strip('%')) for col in intervention_df.columns[3:-1]]  # Skip 'Model', 'Target', 'Intervention', and 'Mean'
        y_values = intervention_df.iloc[0, 3:-1].values  # Get the corresponding metric values

        # Plot the line for each intervention
        plt.plot(x_values, y_values, label=intervention, marker='o')

    # Add labels and title
    plt.title(f'{metric_name} Across Poisoning Percentages for {model_name} (Target Class: {target_class})', fontsize=16)
    plt.xlabel('Poisoning Percentage (%)', fontsize=12)
    plt.ylabel(metric_name, fontsize=12)

    # Display legend
    plt.legend(title='Intervention', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Show plot
    plt.tight_layout()
    plt.show()

# Define a function that will update the plot based on user input
def interactive_plot_separate(metric_name, model_name, target_class):
    if metric_name == 'Accuracy':
        plot_metric_separate(accuracy_df, "Accuracy", model_name, target_class)
    elif metric_name == 'Precision':
        plot_metric_separate(precision_df, "Precision", model_name, target_class)
    elif metric_name == 'Recall':
        plot_metric_separate(recall_df, "Recall", model_name, target_class)
    elif metric_name == 'F1 Score':
        plot_metric_separate(f1_df, "F1 Score", model_name, target_class)
    elif metric_name == 'Failure Detection Rate':
        plot_metric_separate(fdr_df, "Failure Detection Rate", model_name, target_class)  # FDR plot

# Create a dropdown menu for selecting the metric (including FDR)
metric_dropdown = widgets.Dropdown(
    options=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Failure Detection Rate'],
    value='Accuracy',
    description='Metric:',
    disabled=False,
)

# Create a dropdown menu for selecting the model (Random Forest, XGBoost, Neural Network)
model_dropdown = widgets.Dropdown(
    options=['Random Forest', 'XGBoost', 'Neural Network'],
    value='Random Forest',
    description='Model:',
    disabled=False,
)

# Create a dropdown menu for selecting the target class (0 or 1)
target_dropdown = widgets.Dropdown(
    options=[0, 1],
    value=0,
    description='Target Class:',
    disabled=False,
)

# Use the interact function to update the plot dynamically
interact(interactive_plot_separate, metric_name=metric_dropdown, model_name=model_dropdown, target_class=target_dropdown)

interactive(children=(Dropdown(description='Metric:', options=('Accuracy', 'Precision', 'Recall', 'F1 Score', …

<function __main__.interactive_plot_separate(metric_name, model_name, target_class)>